In [1]:
import pyzoom as z
import pandas as pd
from datetime import datetime as date
import time
from UDP.sunstone_package import *
import matplotlib.pyplot as plt
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe

In [ ]:
class pull_zoom_data():
    def __init__(sel, api, secret):
        api = api
        secret = secret
        self.client = z.ZoomClient(api_key=api, api_secret=secret)
        
    def get_events(self,date):
        final_d = {}
        error_date = []
        print(f"adding data {date}")
        try:
            meetings = self.client.raw.get_all_pages('/metrics/meetings', {'type':'past','from': date,
                             'to': date})
        except:
            meetings = self.client.raw.get_all_pages('/metrics/meetings', {'type':'past','from': date,
                             'to': date, 'page_size':300})
            
        dict_ = {}
        for d in meetings['meetings']:
            if d['id'] in dict_.keys():
                rep_id = d['id']
                print(f"more entries for id {rep_id}, ;printing all entries, appending only with most participants")
                temp_df = pd.DataFrame(meetings['meetings'])
                print(temp_df.loc[temp_df.id == d['id'],["id",'topic','duration','participants']])

                old_participants = dict_[rep_id][3]
                new_participants = d['participants']
                if new_participants > old_participants:
                    dict_.update({d['id'] : [d['email'],d['topic'],d['duration'], d['participants']]})
            else:    
                dict_.update({d['id'] : [d['email'],d['topic'],d['duration'], d['participants']]})
        cur_d = {date : dict_}
        final_d.update(cur_d)
        self.participant_dict = final_d.copy()
    
    
    def get_participants_data(self, date):
        self.get_events(date)
        client =  self.client
        first_dict = self.participant_dict.copy()

        final_df_date_list = []
        final_df_meeting_id_list = []
        final_df_meeting_name_list = []
        final_df_student_name_list = []
        final_df_email_list = []
        final_df_duration_list = []

        final_df_meetingduration_list = []
        final_df_meetingparticipation_list = []
        final_df_meetingemail_list = []

        error_meeting_id = []
        print("total date keys in the given dictionary :", len(first_dict))
        iter_ = 0

        for date in first_dict.keys():
            print(f'{iter_ + 1} date is running')
            print("========================================")
            cur_dict = first_dict[date]
            if len(cur_dict) > 0:
                for meeting_id in cur_dict.keys():
                    print(f'meeting id : {meeting_id} for date {date}')
                    cur_meeting_email = cur_dict[meeting_id][0]
                    cur_meeting_name = cur_dict[meeting_id][1]
                    cur_meeting_duration = cur_dict[meeting_id][2]
                    cur_meeting_participants = cur_dict[meeting_id][3]
                    stud = client.raw.get_all_pages(f'/report/meetings/{meeting_id}/participants')#, query = {'page_size':300})
                    cur_all_stud_dict = stud['participants']

                    name_list , email_list, dur_list = [],[],[]
                    try:
                        for i in range(len(cur_all_stud_dict)):
                            user_dict = cur_all_stud_dict[i]
                            name_list.append(user_dict['name'])
                            email_list.append(user_dict['user_email'])
                            dur_list.append(user_dict['duration']/60)


                        # multiply meeting list with len(student data)
                        cur_meeting_email_list = [cur_meeting_email] * len(cur_all_stud_dict)
                        cur_meeting_duration_list = [str(cur_meeting_duration)] * len(cur_all_stud_dict)
                        cur_meeting_participants_list = [str(cur_meeting_participants)] * len(cur_all_stud_dict)

                        cur_meeting_name_list = [cur_meeting_name] * len(cur_all_stud_dict)
                        cur_meeting_id_list = [str(meeting_id)] * len(cur_all_stud_dict)
                        cur_date_list = [str(date)] * len(cur_all_stud_dict)

                        final_df_date_list.extend(cur_date_list)
                        final_df_meeting_id_list.extend(cur_meeting_id_list)
                        final_df_meeting_name_list.extend(cur_meeting_name_list)
                        final_df_student_name_list.extend(name_list)
                        final_df_email_list.extend(email_list)
                        final_df_duration_list.extend(dur_list)

                        final_df_meetingduration_list.extend(cur_meeting_duration_list)
                        final_df_meetingparticipation_list.extend(cur_meeting_participants_list)
                        final_df_meetingemail_list.extend(cur_meeting_email_list)


                    except:
                        error_meeting_id.append(meeting_id)
                        print(f"error in meeting id for date {date} , waiting for 60")
                        time.sleep(60)
            iter_ +=1

        final_dict = {"date":final_df_date_list, "meeting_id":final_df_meeting_id_list ,
                      'meeting_email' : final_df_meetingemail_list,
                      "meeting_name" :final_df_meeting_name_list,
                      'meeting_input_duration':final_df_meetingduration_list,
                      'meeting_participation':final_df_meetingparticipation_list,
                 "student_name" :final_df_student_name_list , "stud_email":  final_df_email_list, "duration_min" : final_df_duration_list }

        return [final_dict,error_meeting_id]

In [ ]:
#pull data for a date
z = pull_zoom_data()
data = z.get_participants_data(date = "2023-05-20")[0]